# Горбунов Иван ИУ6-54б

In [17]:
surname = "Горбунов"

alp = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
w = [1, 42, 21, 21, 34,  6, 44, 26, 18, 44, 38, 26, 14, 43,  4, 49, 45,
        7, 42, 29,  4,  9, 36, 34, 31, 29,  5, 30,  4, 19, 28, 25, 33]

d = dict(zip(alp, w))
variant =  sum([d[el] for el in surname.lower()]) % 40 + 1

print("Задача № 1: ", variant % 3 + 1)
print("Задача № 2: ", variant % 2 + 1 )

Задача № 1:  3
Задача № 2:  1


## Настройка

In [1]:
import os
import shutil
from pyspark.sql import SparkSession

# Настройка окружения для macOS
os.environ["OBJC_DISABLE_INITIALIZE_FORK_SAFETY"] = "YES"
os.environ["PYSPARK_PYTHON"] = "python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python3"
# Путь к Java
java_path = "/Library/Java/JavaVirtualMachines/temurin-17.jdk/Contents/Home" 
if not os.path.exists(java_path):
    java_path = "/Library/Java/JavaVirtualMachines/openjdk-17.jdk/Contents/Home"
os.environ["JAVA_HOME"] = java_path

# Запуск Spark Session
try:
    spark.stop()
except:
    pass

spark = SparkSession.builder \
    .appName("HW4_MachineLearning") \
    .master("local[*]") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

print("Spark MLlib готов к работе!")

26/01/11 14:20:01 WARN Utils: Your hostname, MacBook.local resolves to a loopback address: 127.0.0.1; using 192.168.31.202 instead (on interface en0)
26/01/11 14:20:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/11 14:20:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark MLlib готов к работе!


## Задание 1 - Классификация (Вариант 3: Spambase)

In [2]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import urllib.request

# Скачиваем датасет
url_spam = "https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data"
file_spam = "spambase.data"

if not os.path.exists(file_spam):
    print("Скачивание Spambase...")
    urllib.request.urlretrieve(url_spam, file_spam)

# Загружаем данные
data_spam = spark.read.csv(file_spam, inferSchema=True, header=False)

print(f"Данные загружены: {data_spam.count()} строк")

input_cols = [f"_c{i}" for i in range(57)]
data_spam = data_spam.withColumnRenamed("_c57", "label")

# Подготовка признаков
# Spark ML принимает все признаки в виде одного вектора
assembler = VectorAssembler(inputCols=input_cols, outputCol="features")
final_data = assembler.transform(data_spam).select("features", "label")

# Разделение на обучение и тест (70% / 30%)
train_data, test_data = final_data.randomSplit([0.7, 0.3], seed=42)

# Обучение модели - RandomForest
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=20)
model = rf.fit(train_data)

# Предсказание и Оценка
predictions = model.transform(test_data)

# Оцениваем точность (Accuracy)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print("\nРезультат задания 1 (Классификация)")
print(f"Dataset: Spambase")
print(f"Точность модели (Accuracy): {accuracy:.2%}")
print("Пример предсказаний:")
predictions.select("label", "prediction", "probability").show(5)

Данные загружены: 4601 строк


26/01/11 14:20:07 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.



Результат задания 1 (Классификация)
Dataset: Spambase
Точность модели (Accuracy): 93.15%
Пример предсказаний:
+-----+----------+--------------------+
|label|prediction|         probability|
+-----+----------+--------------------+
|    1|       1.0|[0.11453432509771...|
|    1|       1.0|[0.04224703991056...|
|    1|       1.0|[0.04826249244919...|
|    1|       1.0|[0.04826249244919...|
|    1|       1.0|[0.04649753518424...|
+-----+----------+--------------------+
only showing top 5 rows



## Задание 2 - Регрессия (Вариант 1: Bike Sharing)

In [3]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
import zipfile

# Скачиваем датасет
url_bike = "https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip"
zip_bike = "Bike-Sharing-Dataset.zip"
csv_bike = "hour.csv"

if not os.path.exists(csv_bike):
    print("Скачиваю Bike Sharing Data...")
    urllib.request.urlretrieve(url_bike, zip_bike)
    with zipfile.ZipFile(zip_bike, 'r') as z:
        z.extract(csv_bike)

# Загружаем данные
df_bike = spark.read.csv(csv_bike, header=True, inferSchema=True)

# Подготовка признаков
feature_cols = ['season', 'yr', 'mnth', 'hr', 'holiday', 'workingday', 'weathersit', 'temp', 'hum', 'windspeed']

assembler_reg = VectorAssembler(inputCols=feature_cols, outputCol="features")
final_data_reg = assembler_reg.transform(df_bike).select("features", "cnt")

#Разделение на обучение и тест
train_reg, test_reg = final_data_reg.randomSplit([0.7, 0.3], seed=123)

# Обучение модели (Линейная регрессия)
lr = LinearRegression(featuresCol="features", labelCol="cnt")
lr_model = lr.fit(train_reg)

# Оценка
reg_predictions = lr_model.transform(test_reg)
reg_evaluator = RegressionEvaluator(labelCol="cnt", predictionCol="prediction", metricName="rmse")
rmse = reg_evaluator.evaluate(reg_predictions)

print("\nРезультат задания 2 (Регрессия)")
print(f"Ошибка (RMSE): {rmse:.2f}")
print("Пример предсказаний:")
reg_predictions.select("cnt", "prediction").show(5)

26/01/11 14:20:11 WARN Instrumentation: [4b5e320e] regParam is zero, which might cause numerical instability and overfitting.
26/01/11 14:20:11 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
26/01/11 14:20:11 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
26/01/11 14:20:11 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK



Результат задания 2 (Регрессия)
Ошибка (RMSE): 143.40
Пример предсказаний:
+---+-------------------+
|cnt|         prediction|
+---+-------------------+
| 39|-26.994394850802752|
| 89|  100.1366353796962|
|128|  47.45281320676513|
|173| 60.734487688388626|
| 88| 1.2846049058215208|
+---+-------------------+
only showing top 5 rows



## Задание 3 - Кластеризация (Iris)

In [4]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Скачиваем датасет
url_iris = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
file_iris = "iris.data"

if not os.path.exists(file_iris):
    print("Скачиваю Iris...")
    urllib.request.urlretrieve(url_iris, file_iris)

# Загружаем
df_iris = spark.read.csv(file_iris, inferSchema=True)

iris_features = ["_c0", "_c1", "_c2", "_c3"]

assembler_cl = VectorAssembler(inputCols=iris_features, outputCol="features")
dataset_cl = assembler_cl.transform(df_iris).select("features")

# Обучение K-Means
kmeans = KMeans().setK(3).setSeed(1)
model_cl = kmeans.fit(dataset_cl)

#Предсказание кластеров
predictions_cl = model_cl.transform(dataset_cl)

# оценка качества (Silhouette score)
# Показывает, насколько хорошо разделены кластеры (от -1 до 1)
evaluator_cl = ClusteringEvaluator()
silhouette = evaluator_cl.evaluate(predictions_cl)

print("\nРезультат задания 3 Кластеризация")
print(f"Silhouette Score: {silhouette:.4f}")
print("Центры кластеров:")
centers = model_cl.clusterCenters()
for center in centers:
    print(center)


Результат задания 3 Кластеризация
Silhouette Score: 0.7355
Центры кластеров:
[5.9016129  2.7483871  4.39354839 1.43387097]
[5.006 3.418 1.464 0.244]
[6.85       3.07368421 5.74210526 2.07105263]
